Here we will implement a new library for <br>
<b>Hyperopt : a Tree-structured Prazen Estimator (TPE) </b>

In [1]:
import numpy as np
import pandas as pd
from functools import partial

In [4]:
from sklearn import ensemble
from sklearn import metrics
from sklearn import model_selection
from hyperopt import hp, fmin,tpe,Trials
from hyperopt.pyll.base import scope

In [12]:
def optimize(params,x,y):
    model = ensemble.RandomForestClassifier(**params)
    kf = model_selection.StratifiedKFold(n_splits=5)
    accuracies = []
    for idx in kf.split(X=x,y=y):
        train_idx, test_idx = idx[0], idx[1]
        xtrain = x[train_idx]
        ytrain = y[train_idx]
        xtest = x[test_idx]
        ytest = y[test_idx]
        # fit model for current fold
        model.fit(xtrain, ytrain)
        #create predictions
        preds = model.predict(xtest)
        # calculate and append accuracy
        fold_accuracy = metrics.accuracy_score(
        ytest,
        preds
        )
        accuracies.append(fold_accuracy)
    # return negative accuracy
    return -1 * np.mean(accuracies)


In [13]:
if __name__ == "__main__":
    df = pd.read_csv("input/mobile_train.csv")
    X = df.drop("price_range",axis=1).values
    y = df.price_range.values
    param_space = {
        "max_depth":scope.int(hp.quniform("max_depth",1,15,1)),
        "n_estimators":scope.int(
            hp.quniform("_estimators",100,1500,1)
        ),
        "criterion":hp.choice("criterion",['gini','entropy']),
        "max_features":hp.uniform("max_features",0,1)
    }
    optimization_function = partial(
    optimize,
    x = X,
    y=y
    )
    
    trials = Trials()
    
    hopt = fmin(
        fn = optimization_function,
        space = param_space,
        algo = tpe.suggest,
        max_evals = 15,
        trials = trials
    )
    print(hopt)

100%|████████████████████████████████████████████████████████████| 15/15 [04:41<00:00, 18.80s/trial, best loss: -0.905]
{'_estimators': 771.0, 'criterion': 1, 'max_depth': 7.0, 'max_features': 0.7495050075795785}
